In [25]:
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from tqdm import tqdm
from sklearn.preprocessing import FunctionTransformer
from hyperopt import STATUS_OK, Trials, fmin, hp, tpe
import seaborn as sns 
from scipy import stats
import statsmodels.api as sm
from xgboost import XGBRegressor

from sklearn.ensemble import RandomForestRegressor, ExtraTreesRegressor, AdaBoostRegressor, BaggingRegressor, GradientBoostingRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.linear_model import ElasticNet, Lasso
from sklearn.feature_selection import SelectFromModel
from sklearn.svm import SVR
from sklearn.model_selection import KFold, cross_val_score
from sklearn.pipeline import make_pipeline
from sklearn.decomposition import PCA
from sklearn.preprocessing import RobustScaler
import h2o
from h2o.automl import H2OAutoML
from sklearn.base import BaseEstimator, TransformerMixin, RegressorMixin, clone
from sklearn.neural_network import MLPRegressor
from scipy.stats import norm
import copy
from sklearn.model_selection import KFold
from catboost import CatBoostRegressor, Pool, metrics, cv
import xgboost as xgb
from scipy.stats import gmean
from sklearn.impute import KNNImputer
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import LabelEncoder


In [49]:
train_df =  pd.read_csv('n_train.csv')
test_df  =  pd.read_csv('n_test.csv')
for c in train_df.columns:
    if train_df[c].dtypes != 'float64':
        print(c)

startdate
year
quarter
month
week
dayofyear
season
loc_group
climateregions__climateregion


In [50]:
exclude_cols = ['index', 'startdate']
target=["contest-tmp2m-14d__tmp2m"]
diff_cols = ['contest-pevpr-sfc-gauss-14d__pevpr', 'nmme0-tmp2m-34w__cancm30', 'year']
main_attrs = [c for c in train_df.columns if c not in exclude_cols and c not in target and c not in diff_cols]
print("Main features:", len(main_attrs))

Main features: 257


In [55]:
def train_model_by_X_y(_X, _y, iterations=5000):
    skf = KFold(n_splits=5, random_state=2048, shuffle=True)
    skf.get_n_splits(_X, _y)

    train_index_list = []
    test_index_list = []
    train_x_list = []
    val_x_list = []
    train_y_list = []
    val_y_list = []

    for i, (train_index, test_index) in enumerate(skf.split(_X, _y)):
        train_index_list.append(train_index)
        test_index_list.append(test_index)

        train_x_list.append(_X[train_index, :])
        val_x_list.append(_X[test_index, :])
        train_y_list.append(_y[train_index])
        val_y_list.append(_y[test_index])    
   

    cat_models = []
    for i in range(5):
        train_x, val_x = train_x_list[i], val_x_list[i]
        train_y, val_y = train_y_list[i], val_y_list[i],
        clf = XGBRegressor(n_estimators=2000, tree_method='gpu_hist', gpu_id=0)
        clf.fit(train_x, train_y, eval_set=[(val_x, val_y)], verbose=200, early_stopping_rounds=5)
        cat_models.append(clf)
    
    return cat_models


In [56]:
def cal_train_pred(models, _X):
    preds = []
    for model in models:
        preds.append(model.predict(_X))
    s = 0
    for pred in preds:
        s += pred
    s = s/5
    return s

In [57]:
X = train_df[main_attrs].values
y = np.array(train_df[target[0]].tolist())
cat_models = train_model_by_X_y(X, y)

[0]	validation_0-rmse:10.61557
[200]	validation_0-rmse:0.58227
[400]	validation_0-rmse:0.47419
[600]	validation_0-rmse:0.42053
[800]	validation_0-rmse:0.38950
[1000]	validation_0-rmse:0.36906
[1200]	validation_0-rmse:0.35280
[1400]	validation_0-rmse:0.34101
[1600]	validation_0-rmse:0.33236
[1800]	validation_0-rmse:0.32532
[1999]	validation_0-rmse:0.31953
[0]	validation_0-rmse:10.68490
[200]	validation_0-rmse:0.58146
[400]	validation_0-rmse:0.47278
[600]	validation_0-rmse:0.41902
[800]	validation_0-rmse:0.38903
[1000]	validation_0-rmse:0.36806
[1200]	validation_0-rmse:0.35247
[1400]	validation_0-rmse:0.34109
[1600]	validation_0-rmse:0.33185
[1800]	validation_0-rmse:0.32513
[1999]	validation_0-rmse:0.31958
[0]	validation_0-rmse:10.68028
[200]	validation_0-rmse:0.57873
[400]	validation_0-rmse:0.47009
[600]	validation_0-rmse:0.41739
[800]	validation_0-rmse:0.38754
[1000]	validation_0-rmse:0.36617
[1200]	validation_0-rmse:0.35130
[1400]	validation_0-rmse:0.34015
[1600]	validation_0-rmse:0.3

In [67]:
out_cols = ['icec-2010-1',
 'icec-2010-2',
 'icec-2010-3',
 'icec-2010-4',
 'icec-2010-5',
 'icec-2010-6',
 'icec-2010-7',
 'icec-2010-8',
 'icec-2010-9',
 'icec-2010-10',
 'year',
 'quarter',
 'month_cos',
 'quarter_sin',
 'quarter_cos']
selected_cols = [c for c in main_attrs if c not in out_cols]
len(selected_cols)

243

In [68]:
from sklearn.model_selection import KFold
import xgboost as xgb
from catboost import CatBoostRegressor
import lightgbm as lgb
from scipy.stats import gmean
X_train, X_validation, y_train, y_validation = train_test_split(train_df[selected_cols], train_df[target[0]], train_size=0.66, random_state=42)
X_test = test_df[selected_cols]
clf = XGBRegressor(n_estimators=5000, tree_method='gpu_hist', gpu_id=0)
clf.fit(X_train, y_train, eval_set=[(X_validation, y_validation)], verbose=200, early_stopping_rounds=5)

[0]	validation_0-rmse:10.65291
[200]	validation_0-rmse:0.58582
[400]	validation_0-rmse:0.48281
[600]	validation_0-rmse:0.42957
[800]	validation_0-rmse:0.40123
[1000]	validation_0-rmse:0.38179
[1200]	validation_0-rmse:0.36685
[1400]	validation_0-rmse:0.35718
[1600]	validation_0-rmse:0.34895
[1800]	validation_0-rmse:0.34223
[2000]	validation_0-rmse:0.33702
[2200]	validation_0-rmse:0.33300
[2400]	validation_0-rmse:0.32932
[2600]	validation_0-rmse:0.32643
[2800]	validation_0-rmse:0.32387
[3000]	validation_0-rmse:0.32162
[3200]	validation_0-rmse:0.31992
[3400]	validation_0-rmse:0.31832
[3600]	validation_0-rmse:0.31674
[3800]	validation_0-rmse:0.31554
[4000]	validation_0-rmse:0.31440
[4200]	validation_0-rmse:0.31346
[4400]	validation_0-rmse:0.31248
[4600]	validation_0-rmse:0.31158
[4800]	validation_0-rmse:0.31089
[4999]	validation_0-rmse:0.31023


XGBRegressor(base_score=0.5, booster='gbtree', callbacks=None,
             colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1,
             early_stopping_rounds=None, enable_categorical=False,
             eval_metric=None, gamma=0, gpu_id=0, grow_policy='depthwise',
             importance_type=None, interaction_constraints='',
             learning_rate=0.300000012, max_bin=256, max_cat_to_onehot=4,
             max_delta_step=0, max_depth=6, max_leaves=0, min_child_weight=1,
             missing=nan, monotone_constraints='()', n_estimators=5000,
             n_jobs=0, num_parallel_tree=1, predictor='auto', random_state=0,
             reg_alpha=0, reg_lambda=1, ...)

In [75]:
feature_importances = clf.feature_importances_
feature_names = main_attrs
selected_cols = []
for score, name in sorted(zip(feature_importances, feature_names), reverse=True):
    if score >= 0.01:
        selected_cols.append(name)
    print('{}: {}'.format(name, score))

nmme-tmp2m-56w__ccsm3: 0.46922892332077026
nmme-tmp2m-34w__cancm3: 0.09885124117136002
week_sin: 0.09585106372833252
nmme-tmp2m-34w__ccsm3: 0.06870243698358536
nmme-tmp2m-34w__ccsm4: 0.056847479194402695
nmme-tmp2m-56w__cancm3: 0.021663671359419823
week_cos: 0.01844319887459278
day_of_year_cos: 0.017427567392587662
nmme-tmp2m-34w__cfsv2: 0.016136689111590385
nmme-tmp2m-56w__gfdl: 0.01347532868385315
wind-vwnd-250-2010-5: 0.005844260565936565
contest-wind-vwnd-250-14d__wind-vwnd-250: 0.004909221548587084
wind-hgt-850-2010-7: 0.00456959055736661
nmme-prate-34w__gfdlflora: 0.0043989988043904305
wind-uwnd-250-2010-12: 0.003963656723499298
nmme-tmp2m-56w__ccsm4: 0.0037518583703786135
wind-hgt-500-2010-7: 0.0035682523157447577
nmme-tmp2m-34w__gfdlflora: 0.003184577915817499
wind-vwnd-250-2010-17: 0.002937460318207741
wind-uwnd-250-2010-3: 0.0025088097900152206
wind-uwnd-925-2010-7: 0.0024924378376454115
nmme-tmp2m-56w__cfsv2: 0.002490192884579301
icec-2010-10: 0.002462753327563405
wind-hgt-8

In [72]:
X = train_df[selected_cols].values
y = np.array(train_df[target[0]].tolist())
cat_models = train_model_by_X_y(X, y)

[0]	validation_0-rmse:10.62788
[200]	validation_0-rmse:0.95953
[400]	validation_0-rmse:0.79154
[600]	validation_0-rmse:0.70017
[800]	validation_0-rmse:0.64245
[1000]	validation_0-rmse:0.60434
[1200]	validation_0-rmse:0.57245
[1400]	validation_0-rmse:0.54825
[1600]	validation_0-rmse:0.52974
[1800]	validation_0-rmse:0.51478
[1999]	validation_0-rmse:0.50163
[0]	validation_0-rmse:10.70095
[200]	validation_0-rmse:0.96219
[400]	validation_0-rmse:0.79673
[600]	validation_0-rmse:0.70216
[800]	validation_0-rmse:0.64297
[1000]	validation_0-rmse:0.60249
[1200]	validation_0-rmse:0.57212
[1400]	validation_0-rmse:0.54737
[1600]	validation_0-rmse:0.52825
[1800]	validation_0-rmse:0.51219
[1999]	validation_0-rmse:0.49922
[0]	validation_0-rmse:10.69680
[200]	validation_0-rmse:0.96757
[400]	validation_0-rmse:0.79098
[600]	validation_0-rmse:0.69587
[800]	validation_0-rmse:0.63870
[1000]	validation_0-rmse:0.59727
[1200]	validation_0-rmse:0.56594
[1400]	validation_0-rmse:0.54283
[1600]	validation_0-rmse:0.5

In [74]:
preds = []
for cat_model in tqdm(cat_models):
    preds.append(cat_model.predict(test_df[selected_cols].values))
s = 0
for pred in preds:
    s += pred
s = s/5
df = pd.read_csv('./sample_solution.csv')
df['contest-tmp2m-14d__tmp2m'] = s
df.to_csv('submission_xgboost.csv', index=False)

best_df = pd.read_csv('best_sub.csv')
y_best = best_df[target[0]].values

from numpy import dot
from numpy.linalg import norm

cos_sim = dot(y_best, s)/(norm(y_best)*norm(s))
print("cos_sim with best submission:", cos_sim)

100%|████████████████████████████████████████████████████████████████████████████████████████| 5/5 [00:00<00:00,  6.65it/s]


cos_sim with best submission: 0.984941311974104
